In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("../../../lib")

import sheet_unfolding.sim as sim
from matplotlib.colors import LogNorm

import pyfof

# Goals:
* Run a friends of friends algorithm on a simulation
* Plot particles that are in haloes
* Plot a halo mass function
* Plot displacement fields in Lagrangian space
* Plot halo particles in Lagrangian space


# Run a simulation
this code you don't need to write here

In [ ]:
myic = sim.ic.IC2DCosmo(512, L=100., rs=1.5, sigma8=2.)

mysim = sim.CosmologicalSimulation2d(myic, aic=0.02, dafac_max=0.05, da_max=0.02, ngrid_pm=128)

mysim.integrate_till(1.)

In [ ]:
rho = sim.sim.deposit2d(mysim.pos, 128, myic.L, mode="cic")
plt.imshow(np.clip(rho / np.mean(rho), 1e-2, None).T, origin="lower", norm = LogNorm(vmin=1e-2, vmax=1e2), extent=[0,100.,0.,100.])
plt.title("a = %g" % mysim.a)
plt.xlabel(r"$x$ [Mpc/$h$]", fontsize=14)
plt.ylabel(r"$y$ [Mpc/$h$]", fontsize=14)

# Use a Friends of Friends algorithm to detect haloes
install the module "pyfof". In linux you could do it e.g. with

```pip install pyfof```

Make sure to learn how to install python modules with a single command, because you will need it often.

Read a bit about the friends of friends algorithm. E.g. here: https://swift.dur.ac.uk/docs/FriendsOfFriends/algorithm_description.html

Then find friends of friends groups with a linking length of 0.2 mean particle separations. Discard all groups which have less than 20 members. Then make a density plot of all particles that are in a group with N>=20 members

Note: What we are doing here is not exactly correct, because we are neglecting the periodic boundary conditions. However, let's keep it simple for now ;)

In [ ]:
linking_length = 0.2*np.sqrt(np.mean(100.**2/512.**2))
print(linking_length)

In [ ]:
res = pyfof.friends_of_friends(mysim.pos.reshape(-1,2), linking_length)

### Target Output
![title](solutions/img/haloes_versus_particles.png)

# Plot a mass function
Plot the fof mass function. I.e. the number of objects with a given mass (through a histogram). The mass of a fof-group equals the sum of the masses of the particles in the group. Since all particles have equal mass, you can just multiply the length of the group by the particle mass $m_0$

In [ ]:
m0 = mysim.mass[0,0]

### Target Output
![title](solutions/img/haloes_massfunction.png)

# Lagrangian space

Let's plot the displacement field in Lagrangian space. (This first new exercise is independent of the previous ones).

Lagrangian space is the space of the initial locations of particles. We started with all particles on a grid (myic.qi[...,0:2]). Since we kept all particles in the same order, making a Lagrangian plot can be as simple as an plt.imshow(myparticleproperty, ...)

The displacement field  $\vec{s}$ describes how much a particle moved from its initial location and it is defined so that

\begin{align}
\vec{x}(\vec{q}) = \vec{q} + \vec{s}(\vec{q})
\end{align}

where $\vec{x}$ are the particle positions and $\vec{q}$ their Lagrangian (initial coordinates). Therefore in principle $\vec{s} = \vec{x} - \vec{q}$. However, since we always wrap particle positions periodically between [0,L], you have to undo the periodic wrapping. You can do this by subtracting/adding +-L so that $\vec{s}$ lies in [-L/2, L/2].

Make a plot of each, the x and y components, of $\vec{s}$ in Lag. space. Further, put a quiver plot on top to indicate the displacement by arrows.

In [ ]:
L = 100.
disp = (((mysim.pos - myic.qi) + L/2.) % L) - L/2.

### Target Output
![title](solutions/img/haloes_lagrangian_displacement.png)

# Lagrangian space haloes

Now let's get to the plot I actually want. Plot the halo index in Lagrangian space, so that different haloes will have different colors. Also plot the displacement field on top as a quiver plot.

To get the halo index in Lagrangian space, you will have to mark each particle by the halo index it is part of. For this you will have to rearange a bit the information that you got from the output of pyfof

### Target Output
![title](solutions/img/haloes_lagrangian_protohaloes.png)